In [1]:
import vcf

In [2]:
vcf_reader = vcf.Reader(open('./samtools_out/lsd_2_vars.vcf', 'r'))
counter = 0
for record in vcf_reader:
    print(record)
    print(record.INFO["DP"])
    print(record.INFO["DP4"])
    print(record.INFO)
    print("--------------------")
    counter += 1

print(counter)
    

Record(CHROM=MH893760.2, POS=11566, REF=TTTATTAT, ALT=[TTTAT])
23
[1, 1, 3, 8]
{'INDEL': True, 'IDV': 8, 'IMF': 0.347826, 'DP': 23, 'VDB': 0.579086, 'SGB': -0.676189, 'MQSB': 0.785785, 'MQ0F': 0.173913, 'AF1': 1.0, 'AC1': 1.0, 'DP4': [1, 1, 3, 8], 'MQ': 13, 'FQ': -999.0, 'PV4': [1.0, 1.0, 1.0, 1.0]}
--------------------
Record(CHROM=MH893760.2, POS=12057, REF=GTTTTTTTTTTT, ALT=[GTTTTTTTTTTTTT])
37
[1, 0, 0, 11]
{'INDEL': True, 'IDV': 4, 'IMF': 0.108108, 'DP': 37, 'VDB': 0.118797, 'SGB': -0.676189, 'MQSB': 1.0, 'MQ0F': 0.0810811, 'AF1': 1.0, 'AC1': 1.0, 'DP4': [1, 0, 0, 11], 'MQ': 21, 'FQ': -999.0, 'PV4': [0.0833333, 1.0, 1.0, 1.0]}
--------------------
Record(CHROM=MH893760.2, POS=16010, REF=TAAAAAAAAA, ALT=[TAAAAAAAAAA])
15
[0, 0, 1, 5]
{'INDEL': True, 'IDV': 4, 'IMF': 0.266667, 'DP': 15, 'VDB': 0.258434, 'SGB': -0.616816, 'MQSB': 1.0, 'MQ0F': 0.0, 'AF1': 1.0, 'AC1': 1.0, 'DP4': [0, 0, 1, 5], 'MQ': 33, 'FQ': -999.0}
--------------------
Record(CHROM=MH893760.2, POS=18168, REF=CTTTTTTT